# YOLO

You Only Look Once  

* Real-Time object Detection
* 하나의 사진에서 다양한 Object를 Detection (인식을 한번에!)
* Video에 나와있는 물체들을 실시간 분석
* 속도 중심 - YOLO
* coco data set - 다양한 물체에 대해서 인식을 하기 위해 모아둔 Data Set
  
  
* DarkNet - 리눅스에서 사용되는 FrameWork (YOLO와 잘 궁합이 맞는 빠르고, GPU,CPU 잘 동작하는 Framework)  
 * DarkFlow - Tensorflow를 적용한 Darknet --> 좀 어려움
 * CPU 환경에서밖에 돌아가지 않음


In [5]:
import cv2
import numpy as np

min_confidence = 0.5

net = cv2.dnn.readNet('yolov3.weights','yolov3.cfg')
classes = []
with open("coco.names","r") as f:
    classes = [line.strip() for line in f.readlines()] #coco.names의 파일을 읽어서 저장
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0]-1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0,255,size=(len(classes),3)) #Color 지정 classes length만큼 channel 3개 해서 0,255까지의 랜덤 수를 줌

img = cv2.imread('../source/yolo_01.jpg') #img Read
img = cv2.resize(img,None,fx=0.4,fy=0.4) # Size 는 0.4만큼 줄임
height,width,channels = img.shape

blob = cv2.dnn.blobFromImage(img,0.00392,(416,416),(0,0,0),True,crop=False)
# 320x320 -- 빠른 스피드 but 정확성이 떨어짐
# 419x419 -- 중간지점
# 609x609 -- 정확도는 높음 but 스피드가 느림

net.setInput(blob)
outs = net.forward(output_layers) #갖고 온 데이터를 모델에 로드

class_ids = []
confidences = []
boxes = []

for out in outs:
    for detection in out:
        scores = detection[5:] #detection[0:5] 까지는 위치값이다.
        class_id = np.argmax(scores) #카테고리중 가장 큰 값을 나타내는 것
        confidence = scores[class_id]
        if confidence > min_confidence: 
            #object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            
            # Rectangle Cooridnates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            
            boxes.append([x,y,w,h])
            confidences.append(float(confidence))
            class_ids.append(class_id)
            
indexes = cv2.dnn.NMSBoxes(boxes,confidences,min_confidence,0.4) # --> Boxing 되어 있는 것들을 하나로 (노이즈를 없애는 데 사용)
print(indexes)
font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)): #boxes의 갯수만큼 반복
    if i in indexes:
        x,y,w,h=boxes[i]
        label = str(classes[class_ids[i]])
        print(i, label)
        color = colors[i]
        cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
        cv2.putText(img,label,(x,y+30),font,2,color,1)
        
cv2.imshow("YOLO IMAGE",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[ 6]
 [ 8]
 [ 0]
 [11]
 [12]
 [ 3]
 [ 9]]
0 car
3 car
6 car
8 motorbike
9 person
11 person
12 person


# YOLO_Video

In [8]:
def detectAndDisplay(frame):
    img = cv2.resize(frame,None,fx=0.4,fy=0.4) # Size 는 0.4만큼 줄임
    height,width,channels = img.shape

    blob = cv2.dnn.blobFromImage(img,0.00392,(416,416),(0,0,0),True,crop=False)
    # 320x320 -- 빠른 스피드 but 정확성이 떨어짐
    # 419x419 -- 중간지점
    # 609x609 -- 정확도는 높음 but 스피드가 느림

    net.setInput(blob)
    outs = net.forward(output_layers) #갖고 온 데이터를 모델에 로드

    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:] #detection[0:5] 까지는 위치값이다.
            class_id = np.argmax(scores) #카테고리중 가장 큰 값을 나타내는 것
            confidence = scores[class_id]
            if confidence > min_confidence: 
                #object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle Cooridnates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x,y,w,h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes,confidences,min_confidence,0.4) # --> Boxing 되어 있는 것들을 하나로 (노이즈를 없애는 데 사용)
#     print(indexes)
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)): #boxes의 갯수만큼 반복
        if i in indexes:
            x,y,w,h=boxes[i]
            label = "{}: {:.2f}".format(classes[class_ids[i]],confidences[i] * 100)
            print(i, label)
            color = colors[i]
            cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
            cv2.putText(img,label,(x,y-5),font,1,color,1)
    cv2.imshow("YOLO IMAGE",img)

In [9]:
import cv2
import numpy as np

min_confidence = 0.5

net = cv2.dnn.readNet('yolov3.weights','yolov3.cfg')
classes = []
with open("coco.names","r") as f:
    classes = [line.strip() for line in f.readlines()] #coco.names의 파일을 읽어서 저장
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0]-1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0,255,size=(len(classes),3)) #Color 지정 classes length만큼 channel 3개 해서 0,255까지의 랜덤 수를 줌

img = cv2.imread('../source/yolo_01.jpg')

file_name= '../source/yolo_01.mp4'
cap = cv2.VideoCapture(file_name)
if not cap.isOpened:
    print("error")
    exit(0)
while True:
    ret, frame= cap.read()
    if frame is None:
        print("End Video")
        break
    detectAndDisplay(frame) #Frame 단위로 실행
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


[[4]
 [0]
 [1]
 [3]
 [2]
 [5]
 [8]]
0 car: 99.87
1 car: 99.86
2 car: 99.52
3 car: 99.53
4 car: 99.97
5 car: 98.29
8 car: 93.66
[[4]
 [0]
 [1]
 [2]
 [3]
 [5]
 [7]]
0 car: 99.87
1 car: 99.86
2 car: 99.51
3 car: 99.50
4 car: 99.97
5 car: 98.35
7 car: 93.50
[[4]
 [0]
 [1]
 [2]
 [3]
 [5]
 [7]]
0 car: 99.87
1 car: 99.86
2 car: 99.51
3 car: 99.50
4 car: 99.97
5 car: 98.35
7 car: 93.58
[[4]
 [1]
 [0]
 [2]
 [5]
 [3]
 [6]]
0 car: 99.84
1 car: 99.85
2 car: 99.37
3 car: 99.05
4 car: 99.96
5 car: 99.09
6 car: 78.32
[[4]
 [1]
 [0]
 [2]
 [3]
 [5]
 [6]]
0 car: 99.84
1 car: 99.86
2 car: 99.36
3 car: 99.05
4 car: 99.96
5 car: 99.05
6 car: 78.47
[[4]
 [1]
 [0]
 [2]
 [5]
 [3]
 [6]]
0 car: 99.84
1 car: 99.86
2 car: 99.36
3 car: 99.05
4 car: 99.96
5 car: 99.12
6 car: 78.47
[[ 4]
 [ 0]
 [ 1]
 [ 5]
 [ 2]
 [ 3]
 [10]]
0 car: 99.90
1 car: 99.84
2 car: 99.16
3 car: 99.09
4 car: 99.98
5 car: 99.39
10 car: 56.40
[[ 4]
 [ 0]
 [ 1]
 [ 5]
 [ 2]
 [ 3]
 [10]]
0 car: 99.90
1 car: 99.84
2 car: 99.19
3 car: 99.09
4 car: 9